In [27]:
#Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [28]:
df=pd.read_excel("Data_Sampanis.xlsx",parse_dates=["Datetime UTC"])
df.shape

(55319, 9)

In [29]:
df.head()

,Unnamed: 0,Datetime UTC,Datetime EET(UTC+2/UTC+3),Wind Generation Forecast,Solar Generation Forcast,Load Forcast,Day-Ahead Price,Carbon Futures,Dutch TTF NG Futures
0,0,2016-12-31 22:00:00+00:00,2017-01-01T00:00UTC+02,500.0,0.0,6243,51.10,6.54,19.541
1,1,2016-12-31 23:00:00+00:00,2017-01-01T01:00UTC+02,480.0,0.0,5793,49.41,6.54,19.541
2,2,2017-01-01 00:00:00+00:00,2017-01-01T02:00UTC+02,470.0,0.0,5623,49.42,6.54,19.541
3,3,2017-01-01 01:00:00+00:00,2017-01-01T03:00UTC+02,450.0,0.0,5294,49.45,6.54,19.541
4,4,2017-01-01 02:00:00+00:00,2017-01-01T04:00UTC+02,450.0,0.0,5042,48.97,6.54,19.541


In [30]:
# Sort DataFrame in date order
df.sort_values(by=["Datetime UTC"], inplace=True, ascending=True)
df.head()

,Unnamed: 0,Datetime UTC,Datetime EET(UTC+2/UTC+3),Wind Generation Forecast,Solar Generation Forcast,Load Forcast,Day-Ahead Price,Carbon Futures,Dutch TTF NG Futures
0,0,2016-12-31 22:00:00+00:00,2017-01-01T00:00UTC+02,500.0,0.0,6243,51.10,6.54,19.541
1,1,2016-12-31 23:00:00+00:00,2017-01-01T01:00UTC+02,480.0,0.0,5793,49.41,6.54,19.541
2,2,2017-01-01 00:00:00+00:00,2017-01-01T02:00UTC+02,470.0,0.0,5623,49.42,6.54,19.541
3,3,2017-01-01 01:00:00+00:00,2017-01-01T03:00UTC+02,450.0,0.0,5294,49.45,6.54,19.541
4,4,2017-01-01 02:00:00+00:00,2017-01-01T04:00UTC+02,450.0,0.0,5042,48.97,6.54,19.541


In [31]:
df.dropna(inplace=True)
df.columns

Index(['Unnamed: 0', 'Datetime UTC', 'Datetime EET(UTC+2/UTC+3)',
       'Wind Generation Forecast', 'Solar Generation Forcast', 'Load Forcast',
       'Day-Ahead Price', 'Carbon Futures', 'Dutch TTF NG Futures'],
      dtype='object')

In [32]:
# Add datetime parameters for Datetime UTC
df["Year"] = df["Datetime UTC"].dt.year
df["Month"] = df["Datetime UTC"].dt.month
df["Day"] = df["Datetime UTC"].dt.day
df["Dayofweek"] = df["Datetime UTC"].dt.dayofweek
df["Dayofyear"] = df["Datetime UTC"].dt.dayofyear
df["Hour"]=df["Datetime UTC"].dt.hour
# Drop original date
df.drop("Datetime UTC", axis=1, inplace=True)

In [33]:
df.head()

,Unnamed: 0,Datetime EET(UTC+2/UTC+3),Wind Generation Forecast,Solar Generation Forcast,Load Forcast,Day-Ahead Price,Carbon Futures,Dutch TTF NG Futures,Year,Month,Day,Dayofweek,Dayofyear,Hour
0,0,2017-01-01T00:00UTC+02,500.0,0.0,6243,51.10,6.54,19.541,2016,12,31,5,366,22
1,1,2017-01-01T01:00UTC+02,480.0,0.0,5793,49.41,6.54,19.541,2016,12,31,5,366,23
2,2,2017-01-01T02:00UTC+02,470.0,0.0,5623,49.42,6.54,19.541,2017,1,1,6,1,0
3,3,2017-01-01T03:00UTC+02,450.0,0.0,5294,49.45,6.54,19.541,2017,1,1,6,1,1
4,4,2017-01-01T04:00UTC+02,450.0,0.0,5042,48.97,6.54,19.541,2017,1,1,6,1,2


In [34]:
# Function to return a dataframe of all spikes in a specific year
def thres(year):
    # Get all electricity prices
    prices=df[df["Year"]==year]["Day-Ahead Price"].copy()
    prices.sort_values(ascending=True,inplace=True)
    prices=prices.reset_index()
    neg_thres=prices.loc[int(len(prices)*0.05)][1]
    pos_thres=prices.loc[int(len(prices)*0.95)][1]
    return [neg_thres,pos_thres]

In [35]:
temp={}
for year in range(2016,2023+1):
    temp[year]=thres(year)

In [36]:
temp

{2016: [49.41, 51.1],
 2017: [38.07, 82.0],
 2018: [41.52, 76.2],
 2019: [39.66, 80.65],
 2020: [22.0, 76.03],
 2021: [41.6, 270.33],
 2022: [139.95, 503.86],
 2023: [59.27, 261.22]}

In [37]:
# Function that returns 1 for spike
def is_spike(price,year):
    if (price<temp[year][0] or price>temp[year][1]):
        return 1
    else:
        return 0

### We will add a column to the dataframe, which indicates whether the price is a spike (1) or not (0)

In [38]:
df['Spike'] = df.apply(lambda x: is_spike(x['Day-Ahead Price'],x['Year']), axis=1)

In [39]:
df.head()

,Unnamed: 0,Datetime EET(UTC+2/UTC+3),Wind Generation Forecast,Solar Generation Forcast,Load Forcast,Day-Ahead Price,Carbon Futures,Dutch TTF NG Futures,Year,Month,Day,Dayofweek,Dayofyear,Hour,Spike
0,0,2017-01-01T00:00UTC+02,500.0,0.0,6243,51.10,6.54,19.541,2016,12,31,5,366,22,0
1,1,2017-01-01T01:00UTC+02,480.0,0.0,5793,49.41,6.54,19.541,2016,12,31,5,366,23,0
2,2,2017-01-01T02:00UTC+02,470.0,0.0,5623,49.42,6.54,19.541,2017,1,1,6,1,0,0
3,3,2017-01-01T03:00UTC+02,450.0,0.0,5294,49.45,6.54,19.541,2017,1,1,6,1,1,0
4,4,2017-01-01T04:00UTC+02,450.0,0.0,5042,48.97,6.54,19.541,2017,1,1,6,1,2,0


In [40]:
df['Spike'].sum()

5345

### Now our target is the Spike variable

In [41]:
df1=df.copy()
df1.drop(['Day-Ahead Price'],axis=1,inplace=True)

In [42]:
df1.head()

,Unnamed: 0,Datetime EET(UTC+2/UTC+3),Wind Generation Forecast,Solar Generation Forcast,Load Forcast,Carbon Futures,Dutch TTF NG Futures,Year,Month,Day,Dayofweek,Dayofyear,Hour,Spike
0,0,2017-01-01T00:00UTC+02,500.0,0.0,6243,6.54,19.541,2016,12,31,5,366,22,0
1,1,2017-01-01T01:00UTC+02,480.0,0.0,5793,6.54,19.541,2016,12,31,5,366,23,0
2,2,2017-01-01T02:00UTC+02,470.0,0.0,5623,6.54,19.541,2017,1,1,6,1,0,0
3,3,2017-01-01T03:00UTC+02,450.0,0.0,5294,6.54,19.541,2017,1,1,6,1,1,0
4,4,2017-01-01T04:00UTC+02,450.0,0.0,5042,6.54,19.541,2017,1,1,6,1,2,0


### Split the data into X & Y

In [43]:
X=df1.drop(["Spike","Unnamed: 0","Datetime EET(UTC+2/UTC+3)"],axis=1)
y=df1["Spike"]

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=False)

In [45]:
X_train_sequence=X_train.values.tolist()

In [46]:
X_train.shape

(42911, 11)

In [47]:
y_train_sequence=y_train.values.tolist()

In [48]:
for i in range (len(X_train_sequence)):
    for j in range(11):
        X_train_sequence[i][j]=[X_train_sequence[i][j]]

In [49]:
X_train_sequence1=np.array(X_train_sequence)
y_train_sequence1=np.array(y_train_sequence)

### Create the transformer encoder

In [50]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

### Function to build model

In [51]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [53]:
input_shape=X_train_sequence1.shape[1:]
model=build_model(input_shape,head_size=256,num_heads=4,ff_dim=8,num_transformer_blocks=2,mlp_units=[128],mlp_dropout=0.4,dropout=0.25)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
callbacks=[keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)]
model.fit(X_train_sequence1,y_train_sequence1,validation_split=0.2,epochs=200,batch_size=64,callbacks=callbacks)


Epoch 1/200
537/537 [==============================] - 58s 99ms/step - loss: 3.7190 - accuracy: 0.7569 - val_loss: 1.2742 - val_accuracy: 0.9174
Epoch 2/200
537/537 [==============================] - 51s 95ms/step - loss: 2.3042 - accuracy: 0.8499 - val_loss: 1.2742 - val_accuracy: 0.9174
Epoch 3/200
537/537 [==============================] - 50s 94ms/step - loss: 1.9794 - accuracy: 0.8713 - val_loss: 1.2742 - val_accuracy: 0.9174
Epoch 4/200
537/537 [==============================] - 63s 117ms/step - loss: 1.8227 - accuracy: 0.8816 - val_loss: 1.2742 - val_accuracy: 0.9174
Epoch 5/200
537/537 [==============================] - 48s 89ms/step - loss: 1.7644 - accuracy: 0.8855 - val_loss: 1.2742 - val_accuracy: 0.9174
Epoch 6/200
537/537 [==============================] - 52s 97ms/step - loss: 1.7374 - accuracy: 0.8872 - val_loss: 1.2742 - val_accuracy: 0.9174
Epoch 7/200
537/537 [==============================] - 51s 95ms/step - loss: 1.7241 - accuracy: 0.8881 - val_loss: 1.2742 - val_a

In [54]:
X_test_sequence=X_test.values.tolist()
y_test_sequence=y_test.values.tolist()
X_test_sequence1=np.array(X_test_sequence)
y_test_sequence1=np.array(y_test_sequence)

In [55]:
predictions=model.predict(X_test_sequence1)

336/336 [==============================] - 6s 16ms/step


In [56]:
predictions

array([[-1119.5999 ],
       [ -990.5022 ],
       [ -888.1051 ],
       ...,
       [ -939.62396],
       [ -860.8934 ],
       [ -797.35   ]], dtype=float32)